# [텍스트 분류하기]

## 1. 베이지안 필터(Baysian filter)
학습을 시킬수록 필터의 분류 능력이 오르는 특징을 가지고 있다.  
때문에 메일 서비스에서 스팸 메일을 구분하거나/커뮤니티 사이트에서 스팸 글을 구분할 때 많이 사용된다. 이외에도 문장의 카데고리 분류에도 사용된다.  

*교사학습, 비교사학습, 강화학습 중 "교사학습"에 해당됨

### 1) 이론

#### (1) 베이즈 정리: 조건부 확률과 관련된 이론으로, 토머스 베이즈에 의해 정리된 이론  

P(B|A)=P(A|B)P(B) / P(A)  
* P(A): A가 일어날 확률
* P(B): B가 일어날 확률(사전확률)
* P(A|B): B가 일어난 후 A가 일어날 확률(조건부확률. 사후확률)
* P(B|A): A가 일어난 후 B가 일어날 확률(조건부확률. 사전확률)


#### (2) 조건부확률
<어떤 A라는 사건이 일어났다는 조건>에서 <다른 사건 B가 일어날 확률>을 나타냄  

ex)  
* 비가 내릴 확률: P(비)
* 교통사고가 발생활 확률: P(교통사고)
* 비가 내리는 날에 교통사고가 발생할 확률: P(교통사고|비)

### 2) 알고리즘

#### 나이브 베이즈 분류(Naive Bayes classifier)
베이즈 정리를 사용한 분류 방법.  
텍스트 내부에서 단어 출현 비율을 조사→이를 기반으로 해당 텍스트를 어떤 카테고리로 분류하는 것이 적합한지 조사  

실제로 판정을 할 때 P(B|A)는 1개의 확률이 아니라 여러 개의 카테고리 중에 어떤 카테고리에 속할 확률이 가장 큰지를 나타내는 정보이다.  
따라서 베이즈 정리의 분모에 있는 P(A)는 입력 텍스트가 주어질 확률이다.  
다만 어떤 카테고리를 판정하든 같은 입력 텍스트가 주어지는 것이므로 같은 값으로 생각하면 된다.  
정리하면,  
P(B|A)=P(B)P(A|B)가 된다.  

결국 나이브 베이즈 분류는  
단순한 출현율=단어의 출현횟수/카테고리 전체 단어 수  
P(A|B)=P(B) / P(B|A)  
가 된다.

In [2]:
import math, sys
from konlpy.tag import Twitter

In [4]:
class BayesianFilter:
    """ 베이지안 필터 """
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 횟수 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
        
        
        
    # 1. 형태소 분석하기
    def split(self, text):
        return text.split()
        results = []
        twitter = Twitter()
        # 단어의 기본형 사용
        malist = twitter.pos(text, norm=True, stem=True)
        for word in malist:
            # 어미/조사/구두점 등은 대상에서 제외 
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])
        return results
    
    
    
    # 2. 단어와 카테고리의 출현 횟수 세기 
    def inc_word(self, word, category): #단어 횟수 세기
        # 단어를 카테고리에 추가하기
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
        
    def inc_category(self, category): #카테고리 횟수 세기
        # 카테고리 계산하기
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1
    
    
    
    # 3. 텍스트 학습하기 
    def fit(self, text, category): #텍스트를 형태소로 분할하고, 카테고리와 단어를 연결함
        """ 텍스트 학습 """
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word, category)
        self.inc_category(category)
    
    
    
    # 4. 단어 리스트에 점수 매기기
    def score(self, words, category):
        #확률을 곱할 때 값이 너무 작으면 다운플로가 발생할 수 있어 log를 이용함
        score = math.log(self.category_prob(category)) 
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    
    
    # 5. 예측하기 
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize 
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list
    
    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
        
    # 카테고리 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
        
        
        
    # 6. 카테고리 내부의 단어 출현 비율 계산 
    def word_prob(self, word, category):
        #단어 출현률을 계산할 때 학습사전(word_dict)에 없는 단어가 나오면 카테고리의 확률이 0이 되어버려 1을 더해 활용
        n = self.get_word_count(word, category) + 1 # ---(※6a)
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d

In [6]:
### 예시
bf= BayesianFilter()

# 텍스트 학습
bf.fit("파격 세일 - 오늘까지만 30% 할인", "광고")
bf.fit("쿠폰 선물 & 무료 배송", "광고")
bf.fit("현데계 백화점 세일", "광고")
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고")
bf.fit("인기 제품 기간 한정 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고","중요")
bf.fit("계약 잘 부탁드립니다","중요")
bf.fit("회의 일정이 등록되었습니다.","중요")
bf.fit("오늘 일정이 없습니다.","중요")

# 예측
pre, scorelist = bf.predict("재고 정리 할인, 무료 배송")
print("결과 =", pre)
print(scorelist)

결과 = 광고
[('중요', -20.544606748320554), ('광고', -19.942524744665512)]


## 2. MLP
다층 퍼셉트론(Multi Layer Perceptron, MLP). 입력층과 출력층 사이에 각각 전체 결합하는 은닉층을 넣은 뉴럴 네트워크  
텍스트 데이터를 숫자로 표현할 수 있는 벡터로 변환해서 사용한다. 

### 1) 텍스트 데이터를 고정 길이의 벡터로 변환하는 방법
단어 하나하나에 ID를 부여하고, ID의 출현 빈도와 정렬 순서를 기반으로 벡터를 만든다. = Bow 이용
* Bow: Bag-of-words. 글에 어떠한 단어가 있는지를 수치로 나타내는 방법

In [11]:
from konlpy.tag import Twitter

In [12]:
twitter=Twitter()
st=twitter.pos("몇 번을 쓰러지더라도 몇 번을 무너지더라도 다시 일어나라", stem=True, norm= True)
print(st)

[('몇', 'Noun'), ('번', 'Noun'), ('을', 'Josa'), ('쓰러지다', 'Verb'), ('몇', 'Noun'), ('번', 'Noun'), ('을', 'Josa'), ('무너지다', 'Verb'), ('다시', 'Noun'), ('일어나다', 'Verb')]


C:\Anaconda3\envs\tensorflow\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


#### 예시(몇 번을 쓰러지더라고 몇 번을 무너지더라도 다시 일어나라)에 Bow 적용하기
#형태소-ID-출현 횟수
#몇-1-2
#번-2-2
#을-3-2
#쓰러지다-4-1
#무너지다-5-1
#다시-6-1
#일어나다-7-1

### 2) 텍스트 분류하기
[1] 텍스트에서 불필요한 품사를 제거한다.  
[2] 사전을 기반으로 단어를 숫자로 변환한다.  
[3] 파일 내부의 단어 출현 비율을 계산한다.  
[4] 데이터를 학습시킨다.  
[5] 테스트 데이터를 넣어 성공률을 확인한다.

### 3) 단어를 ID로 변환하고 출현 횟수 구하기

In [13]:
import os, glob, json

In [20]:
root_dir = "./newstext"
dic_file = "/word-dic.json"
data_file = "/data.json"
data_file_min = "/data-mini.json"

# 1.어구를 자르고 ID로 변환하기
word_dic = { "_MAX": 0 }

def text_to_ids(text):
    text = text.strip()
    words = text.split(" ")
    result = []
    for n in words:
        n = n.strip()
        if n == "": continue
        if not n in word_dic:
            wid = word_dic[n] = word_dic["_MAX"]
            word_dic["_MAX"] += 1
            print(wid, n)
        else:
            wid = word_dic[n]
        result.append(wid)
    print(result)
    return result



# 2. 파일을 읽고 고정 길이의 배열 리턴하기
def file_to_ids(fname):
    with open(fname, "r") as f:
        text = f.read()
        return text_to_ids(text)
    
    
    
# 3. 딕셔너리에 단어 모두 등록하기
def register_dic():
    files = glob.glob(root_dir+"/*/*.wakati", recursive=True)
    for i in files:
        file_to_ids(i)

        
        
# 4. 파일 내부의 단어 세기
def count_file_freq(fname):
    cnt = [0 for n in range(word_dic["_MAX"])]
    with open(fname,"r") as f:
        text = f.read().strip()
        ids = text_to_ids(text)
        for wid in ids:
            cnt[wid] += 1
    return cnt

#카테고리마다 파일 읽어 들이기
def count_freq(limit = 0):
    X = []
    Y = []
    max_words = word_dic["_MAX"]
    cat_names = []
    for cat in os.listdir(root_dir):
        cat_dir = root_dir + "/" + cat
        if not os.path.isdir(cat_dir): continue
        cat_idx = len(cat_names)
        cat_names.append(cat)
        files = glob.glob(cat_dir+"/*.wakati")
        i = 0
        for path in files:
            print(path)
            cnt = count_file_freq(path)
            X.append(cnt)
            Y.append(cat_idx)
            if limit > 0:
                if i > limit: break
                i += 1
    return X,Y


# 5. 단어 딕셔너리 만들기
if os.path.exists(dic_file):
     word_dic = json.load(open(dic_file))
else:
    register_dic()
    json.dump(word_dic, open(dic_file,"w"))
    
    
    
# 6. 벡터를 파일로 출력하기
# 테스트 목적의 소규모 데이터 만들기
X, Y = count_freq(20)
json.dump({"X": X, "Y": Y}, open(data_file_min,"w"))

# 전체 데이터를 기반으로 데이터 만들기
X, Y = count_freq()
json.dump({"X": X, "Y": Y}, open(data_file,"w"))

print("ok")

ok


### 4) MLP로 텍스트 분류하기

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import json

In [8]:
max_words = 56681 # 입력 단어 수: word-dic.json 파일 참고
nb_classes = 9    # 9개의 카테고리
batch_size = 64 
nb_epoch = 20

# 1. MLP 모델 생성하기
def build_model():
    model = Sequential()
    model.add(Dense(512, input_shape=(max_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model



# 2. 데이터 읽어 들이기
data = json.load(open("./data-mini.json"))
#data = json.load(open("./newstext/data.json"))
X = data["X"] # 텍스트를 나타내는 데이터
Y = data["Y"] # 카테고리 데이터



# 3. 학습하기
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
Y_train = np_utils.to_categorical(Y_train, nb_classes)
model = KerasClassifier(build_fn=build_model, nb_epoch=nb_epoch, batch_size=batch_size)
model.fit(X_train, Y_train)
print(len(X_train),len(Y_train))



# 4. 예측하기
y = model.predict(X_test)
ac_score = metrics.accuracy_score(Y_test, y)
cl_report = metrics.classification_report(Y_test, y)
#print("정답률 =", ac_score)
#print("리포트 =\n", cl_report)
print('ok')

ok
